In [ ]:
import torch

In [11]:
import numpy as np
import pandas as pd
import os
from scipy.io import wavfile
from scipy.signal import stft
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
import librosa as lr
from spafe.features.gfcc import gfcc
from sklearn.preprocessing import StandardScaler
import time
import torch
import torch.nn as nn
from torchvision.models import resnet50
from PIL import Image
import io
from tqdm import tqdm
import time
from spafe.features.gfcc import gfcc

In [ ]:
def get_spectrogram_as_image(mel_spec_segment,output_path,sample_rate):
    plt.figure(figsize=(3, 3))
    lr.display.specshow(mel_spec_segment, x_axis='time', y_axis='mel', sr=sample_rate, hop_length=512, cmap='viridis')
    plt.axis('off')
    plt.tight_layout()

    # Save the plot to a BytesIO object
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png')
    img_buf.seek(0)
    plt.close()

    # Create a PIL Image object from the BytesIO buffer
    img = Image.open(img_buf)
    height, width = img.size
    left=width/20.6
    top=height/20.6
    right=width-left
    bottom=height-top
    img=img.crop((left, top, right, bottom))
    img.save(output_path)

In [ ]:
def mel_segment(input_path,output_path):
    wav_files_no = os.listdir(input_path)
    for file_name in tqdm(wav_files_no,desc="Processing"):
        time.sleep(0.1)
        audio_data, sample_rate = lr.load(os.path.join(input_path,file_name))
        mel_spectrogram = lr.feature.melspectrogram(y=audio_data, sr= sample_rate, n_fft=1024, hop_length=256, n_mels=64, power=2)
        mel_spec_db = lr.power_to_db(mel_spectrogram, ref=np.max)
        
        
        segment_hop_length = 86
        num_segments = int((mel_spec_db.shape[1] - segment_hop_length) / segment_hop_length)
        for i in range(num_segments):
            start_idx = i * segment_hop_length
            end_idx = start_idx + 2*segment_hop_length
            mel_spec_segment = mel_spec_db[:, start_idx:end_idx]
            file_output=file_name.split('.')[0]+"({})".format(i)+".png"
            print(file_output)
            save_point=os.path.join(output_path,file_output)
            get_spectrogram_as_image(mel_spec_segment,save_point,sample_rate)

In [ ]:
drive=["E:\\","F:\\"]
# folders=["0_dB_fan\\fan","6_dB_fan\\fan","-6_dB_fan\\fan","0_dB_pump\\pump","6_dB_pump\\pump","-6_dB_pump\\pump",
#          "0_dB_valve\\valve","6_dB_valve\\valve","-6_dB_valve\\valve","0_dB_slider\\slider","6_dB_slider\\slider",
#          "-6_dB_slider\\slider"]
#mach_id=["id_00","id_02","id_04","id_06"]
status=["normal","abnormal"]
folders=["0_dB_valve\\valve"]
mach_id=["id_04"]
# status=["normal"]

In [ ]:
for fold in folders:
    for m_id in mach_id:
        for st in status:
            input_path=os.path.join(drive[0]+fold,m_id,st)
            output_path=os.path.join(drive[1]+fold,m_id,st)
            mel_segment(input_path,output_path)

In [ ]:
file_path="E:\\0_dB_fan\\fan\\id_00\\abnormal\\00000000.wav"
y, sr = lr.load(file_path, sr=None)
D = lr.amplitude_to_db(lr.stft(y), ref=np.max)

# Plot the STFT
plt.figure(figsize=(12, 4))
lr.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('STFT of {}'.format(file_path))
plt.show()

In [ ]:
file_path="E:\\0_dB_fan\\fan\\id_00\\normal\\00000000.wav"
y, sr = lr.load(file_path, sr=None)
D = lr.amplitude_to_db(lr.stft(y), ref=np.max)

# Plot the STFT
plt.figure(figsize=(12, 4))
lr.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
#plt.colorbar(format='%+2.0f dB')
#plt.title('STFT of {}'.format(file_path))
plt.axis('off')
plt.show()

In [ ]:
def get_STFT_spectrogram(input_path,output_path):
    y, sr = lr.load(input_path, sr=None)
    D = lr.amplitude_to_db(lr.stft(y), ref=np.max)
    plt.figure(figsize=(12, 4))
    lr.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
    plt.axis('off')
    plt.savefig(output_path,format='jpeg')
    plt.close()
    img = Image.open(output_path)
    height, width = img.size
    left=height/10.041
    top=width/9.195
    right=height-left
    left=left+30
    bottom=width-top
    top=top+4
    img=img.crop((left, top, right, bottom))
    img.save(output_path)

In [ ]:
input_loc="E:\\0_dB_fan\\valve\\id_06\\"
output_loc="F:\\STFT\\-6_dB_valve\\id_06\\"
input_loc_nor=input_loc+"normal\\"
input_loc_abnor=input_loc+"abnormal\\"
output_loc_nor=output_loc+"normal\\"
output_loc_abnor=output_loc+"abnormal\\"

In [ ]:
for file_name in tqdm(os.listdir(input_loc_nor),desc="Processing"):
    time.sleep(0.01)
    save_point=output_loc_nor+file_name.split(".")[0]+".jpg"
    get_STFT_spectrogram(os.path.join(input_loc_nor,file_name),save_point)
for file_name in tqdm(os.listdir(input_loc_abnor),desc="Processing"):
    time.sleep(0.01)
    save_point=output_loc_abnor+file_name.split(".")[0]+".jpg"
    get_STFT_spectrogram(os.path.join(input_loc_abnor,file_name),save_point)

In [ ]:
input_tag=["0_dB_fan\\fan\\","6_dB_fan\\fan\\","-6_dB_fan\\fan\\","0_dB_valve\\valve\\",
          "6_dB_valve\\valve\\","-6_dB_valve\\valve\\","0_dB_slider\\slider\\","6_dB_slider\\slider\\",
          "-6_dB_slider\\slider\\","0_dB_pump\\pump\\","6_dB_pump\\pump\\","-6_dB_pump\\pump\\"]
id_no=["id_00\\","id_02\\","id_04\\","id_06\\"]
output_tag=["0_dB_fan\\","6_dB_fan\\","-6_dB_fan\\","0_dB_valve\\","6_dB_valve\\","-6_dB_valve\\","0_dB_slider\\",
           "6_dB_slider\\","-6_dB_slider\\","0_dB_pump\\","6_dB_pump\\","-6_dB_pump\\",]
direc_inp=[]
for inp in input_tag:
    for mac in id_no:
        direc_inp.append("E:\\"+inp+mac)
direc_out=[]
for out in output_tag:
    for mac in id_no:
        direc_out.append("F:\\STFT\\"+out+mac)

In [ ]:
def read_sound_files(input_loc,output_loc):
    input_loc_nor=input_loc+"normal\\"
    input_loc_abnor=input_loc+"abnormal\\"
    output_loc_nor=output_loc+"normal\\"
    output_loc_abnor=output_loc+"abnormal\\"
    for file_name in tqdm(os.listdir(input_loc_nor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_nor+file_name.split(".")[0]+".jpg"
        get_STFT_spectrogram(os.path.join(input_loc_nor,file_name),save_point)
    for file_name in tqdm(os.listdir(input_loc_abnor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_abnor+file_name.split(".")[0]+".jpg"
        get_STFT_spectrogram(os.path.join(input_loc_abnor,file_name),save_point)

In [ ]:
for i in range(len(direc_out)-1):
    read_sound_files(direc_inp[i+1],direc_out[i+1])

In [ ]:
def read_sound_files_MFCC(input_loc,output_loc):
    input_loc_nor=input_loc+"normal\\"
    input_loc_abnor=input_loc+"abnormal\\"
    output_loc_nor=output_loc+"normal\\"
    output_loc_abnor=output_loc+"abnormal\\"
    for file_name in tqdm(os.listdir(input_loc_nor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_nor+file_name.split(".")[0]+".jpg"
        get_MFCC_spectrogram(os.path.join(input_loc_nor,file_name),save_point)
    for file_name in tqdm(os.listdir(input_loc_abnor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_abnor+file_name.split(".")[0]+".jpg"
        get_MFCC_spectrogram(os.path.join(input_loc_abnor,file_name),save_point)

In [ ]:
def get_MFCC_spectrogram(input_path,output_path):
    y, sr = lr.load(input_path, sr=None)
    mfccs = lr.feature.mfcc(y=y, sr=sr, n_mfcc=200)
    plt.figure(figsize=(12, 4))
    lr.display.specshow(mfccs, x_axis='time', sr=sr, cmap='viridis', hop_length=512)
    plt.axis('off')
    plt.savefig(output_path,format='jpeg')
    plt.close()
    img = Image.open(output_path)
    height, width = img.size
    left=height/10.041
    top=width/9.195
    right=height-left
    left=left+30
    bottom=width-top
    top=top+4
    img=img.crop((left, top, right, bottom))
    img.save(output_path)

In [ ]:
input_tag=["0_dB_fan\\fan\\","6_dB_fan\\fan\\","-6_dB_fan\\fan\\","0_dB_valve\\valve\\",
          "6_dB_valve\\valve\\","-6_dB_valve\\valve\\","0_dB_slider\\slider\\","6_dB_slider\\slider\\",
          "-6_dB_slider\\slider\\","0_dB_pump\\pump\\","6_dB_pump\\pump\\","-6_dB_pump\\pump\\"]
id_no=["id_00\\","id_02\\","id_04\\","id_06\\"]
output_tag=["0_dB_fan\\","6_dB_fan\\","-6_dB_fan\\","0_dB_valve\\","6_dB_valve\\","-6_dB_valve\\","0_dB_slider\\",
           "6_dB_slider\\","-6_dB_slider\\","0_dB_pump\\","6_dB_pump\\","-6_dB_pump\\",]
direc_inp=[]
for inp in input_tag:
    for mac in id_no:
        direc_inp.append("E:\\"+inp+mac)
direc_out=[]
for out in output_tag:
    for mac in id_no:
        direc_out.append("F:\\MFCC\\"+out+mac)

In [ ]:
for i in range(len(direc_out)-45):
    print("*********************************")
    print("Iteration:{} for {}".format(i+45,direc_out[i+45]))
    print("*********************************")
    read_sound_files_MFCC(direc_inp[i+45],direc_out[i+45])

In [3]:
def read_sound_files_GFCC(input_loc,output_loc):
    input_loc_nor=input_loc+"normal\\"
    input_loc_abnor=input_loc+"abnormal\\"
    output_loc_nor=output_loc+"normal\\"
    output_loc_abnor=output_loc+"abnormal\\"
    for file_name in tqdm(os.listdir(input_loc_nor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_nor+file_name.split(".")[0]+".jpg"
        get_GFCC_spectrogram(os.path.join(input_loc_nor,file_name),save_point)
    for file_name in tqdm(os.listdir(input_loc_abnor),desc="Processing"):
        time.sleep(0.01)
        save_point=output_loc_abnor+file_name.split(".")[0]+".jpg"
        get_GFCC_spectrogram(os.path.join(input_loc_abnor,file_name),save_point)

In [4]:
def get_GFCC_spectrogram(input_path,output_path):
    y, sr = lr.load(input_path, sr=None)
    mfccs = lr.feature.mfcc(y=y, sr=sr, n_mfcc=200)
    gfccs = gfcc(y, fs = sr, nfilts = 200, num_ceps = 100)
    plt.figure(figsize=(12, 4))
    lr.display.specshow(gfccs, x_axis='time', sr=sr, cmap='viridis')
    plt.axis('off')
    plt.savefig(output_path,format='jpeg')
    plt.close()
    img = Image.open(output_path)
    height, width = img.size
    left=height/10.041
    top=width/9.195
    right=height-left
    left=left+30
    bottom=width-top
    top=top+4
    img=img.crop((left, top, right, bottom))
    img.save(output_path)

In [5]:
input_tag=["0_dB_fan\\fan\\","6_dB_fan\\fan\\","-6_dB_fan\\fan\\","0_dB_valve\\valve\\",
          "6_dB_valve\\valve\\","-6_dB_valve\\valve\\","0_dB_slider\\slider\\","6_dB_slider\\slider\\",
          "-6_dB_slider\\slider\\","0_dB_pump\\pump\\","6_dB_pump\\pump\\","-6_dB_pump\\pump\\"]
id_no=["id_00\\","id_02\\","id_04\\","id_06\\"]
output_tag=["0_dB_fan\\","6_dB_fan\\","-6_dB_fan\\","0_dB_valve\\","6_dB_valve\\","-6_dB_valve\\","0_dB_slider\\",
           "6_dB_slider\\","-6_dB_slider\\","0_dB_pump\\","6_dB_pump\\","-6_dB_pump\\",]
direc_inp=[]
for inp in input_tag:
    for mac in id_no:
        direc_inp.append("E:\\"+inp+mac)
direc_out=[]
for out in output_tag:
    for mac in id_no:
        direc_out.append("F:\\GFCC\\"+out+mac)

In [6]:
k=47
count=0
for i in range(len(direc_out)-k):
    print("*********************************")
    print("Iteration:{} for {}".format(i+k,direc_out[i+k]))
    print("*********************************")
    if count == 4:
        break
    read_sound_files_GFCC(direc_inp[i+k],direc_out[i+k])
    count=count+1

*********************************
Iteration:47 for F:\GFCC\-6_dB_pump\id_06\
*********************************


Processing: 100%|██████████| 102/102 [00:16<00:00,  6.09it/s]
